In [1]:
import pandas as pd
import sqlite3

In [2]:
connect = sqlite3.connect('../data/checking-logs.sqlite')

## Обьединение двух таблиц 

In [3]:
query = """
    CREATE TABLE datamart AS
    SELECT 
        c.uid, 
        c.labname, 
        MIN(timestamp) as first_commit_ts, 
        MIN(p.datetime) as first_view_ts
    FROM checker c
    FULL JOIN pageviews p ON c.uid = p.uid
    WHERE c.status = 'ready'
        AND c.numTrials = 1
        AND c.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
        AND c.uid LIKE 'user_%'
    GROUP BY c.uid, c.labname;
"""
cursor = connect.cursor()
cursor.execute(query)
connect.commit()

datamart = pd.io.sql.read_sql('SELECT * FROM datamart', connect)

datamart['first_commit_ts'] = pd.to_datetime(datamart['first_commit_ts'])
datamart['first_view_ts'] = pd.to_datetime(datamart['first_view_ts'])
datamart

,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136
...,...,...,...,...
135,user_8,laba04s,2020-04-19 10:22:35.761944,NaT
136,user_8,laba05,2020-05-02 13:28:07.705193,NaT
137,user_8,laba06,2020-05-16 17:56:15.755553,NaT
138,user_8,laba06s,2020-05-16 20:01:07.900727,NaT


In [4]:
#cursor = connect.cursor()
#cursor.execute('DROP TABLE IF EXISTS control')
#connect.commit()

In [5]:
datamart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              140 non-null    object        
 1   labname          140 non-null    object        
 2   first_commit_ts  140 non-null    datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 4.5+ KB


## Создаем тестовый и контрольный фреймы на основе datamart

In [6]:
test = datamart[datamart['first_view_ts'].notna()].copy()
control = datamart[datamart['first_view_ts'].isna()].copy()
test.head()

,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136


In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, 0 to 114
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              59 non-null     object        
 1   labname          59 non-null     object        
 2   first_commit_ts  59 non-null     datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.3+ KB


In [8]:
control['first_view_ts'] = pd.to_datetime(test['first_view_ts'].mean())
control

,uid,labname,first_commit_ts,first_view_ts
12,user_11,laba05,2020-05-03 21:06:55.970293,2020-04-27 00:40:05.761783552
13,user_11,project1,2020-05-03 23:45:33.673409,2020-04-27 00:40:05.761783552
14,user_12,laba04,2020-04-18 17:07:51.767358,2020-04-27 00:40:05.761783552
15,user_12,laba04s,2020-04-26 15:42:38.070593,2020-04-27 00:40:05.761783552
16,user_12,laba05,2020-05-03 08:39:25.174316,2020-04-27 00:40:05.761783552
...,...,...,...,...
135,user_8,laba04s,2020-04-19 10:22:35.761944,2020-04-27 00:40:05.761783552
136,user_8,laba05,2020-05-02 13:28:07.705193,2020-04-27 00:40:05.761783552
137,user_8,laba06,2020-05-16 17:56:15.755553,2020-04-27 00:40:05.761783552
138,user_8,laba06s,2020-05-16 20:01:07.900727,2020-04-27 00:40:05.761783552


In [9]:
control.info()

<class 'pandas.core.frame.DataFrame'>
Index: 81 entries, 12 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              81 non-null     object        
 1   labname          81 non-null     object        
 2   first_commit_ts  81 non-null     datetime64[ns]
 3   first_view_ts    81 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 3.2+ KB


## Сохраняем в SQL

In [10]:
test.to_sql('test', connect)
control.to_sql('control', connect)

81

In [11]:
control = pd.io.sql.read_sql('SELECT * FROM control', connect)
control

,index,uid,labname,first_commit_ts,first_view_ts
0,12,user_11,laba05,2020-05-03 21:06:55.970293,2020-04-27 00:40:05.761783
1,13,user_11,project1,2020-05-03 23:45:33.673409,2020-04-27 00:40:05.761783
2,14,user_12,laba04,2020-04-18 17:07:51.767358,2020-04-27 00:40:05.761783
3,15,user_12,laba04s,2020-04-26 15:42:38.070593,2020-04-27 00:40:05.761783
4,16,user_12,laba05,2020-05-03 08:39:25.174316,2020-04-27 00:40:05.761783
...,...,...,...,...,...
76,135,user_8,laba04s,2020-04-19 10:22:35.761944,2020-04-27 00:40:05.761783
77,136,user_8,laba05,2020-05-02 13:28:07.705193,2020-04-27 00:40:05.761783
78,137,user_8,laba06,2020-05-16 17:56:15.755553,2020-04-27 00:40:05.761783
79,138,user_8,laba06s,2020-05-16 20:01:07.900727,2020-04-27 00:40:05.761783


In [12]:
connect.close()